In [1]:
#Install the necessary modules.
import pandas as pd
from thunderpack import ThunderReader 
import pandas as pd
import random
import zipfile
import io
import tempfile
from tqdm import tqdm

In [2]:
#This part is importing the metadata from the hospital notes into dfs.
df_0 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2010_metadata.csv')
df_1 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2011_metadata.csv')
df_2 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2012_metadata.csv')
df_3 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2013_metadata.csv')
df_4 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2014_metadata.csv')
df_5 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2015_metadata.csv')
df_6 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2016_metadata.csv')
df_7 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2017_metadata.csv')
df_8 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2018_metadata.csv')
df_9 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2019_metadata.csv')
df_10 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2020_metadata.csv')
df_11 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2021_metadata.csv')
df_12 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2022_metadata.csv')
df_13 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2023_metadata.csv')
df_14 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2024_metadata.csv')

dfs = pd.concat([df_0,df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10, df_11, df_12, df_13, df_14], ignore_index=True)  # List of DataFrames
print(len(dfs))
dfs.head()

25381614


,BDSPPatientID,NoteTypeFull,Service,CreateDate,DeidentifiedName
0,150641617,Letter,General Medicine/Primary Care,20100603,Notes_1130500157_1156166680_20100603.txt
1,150641617,Progress Note,General Medicine/Primary Care,20100812,Notes_1130500157_1156166681_20100812.txt
2,150641617,Progress Note,Emergency,20100813,Notes_1130500157_1156166682_20100813.txt
3,150641617,Initial Note,Psychiatry,20100813,Notes_1130500157_1156166683_20100813.txt
4,150641617,Telephone,Emergency,20100813,Notes_1130500157_1156166684_20100813.txt


In [3]:
#Data filtering of note metadata.
#Changing the date to datetime format.
dfs['CreateDate'] = pd.to_datetime(dfs['CreateDate'], format='%Y%m%d')

#We will use all of the data we have for BIDMC which consists of 2010-2024. 
start_date = pd.Timestamp('2010-01-01')
end_date = pd.Timestamp('2024-12-31') 
complete_df_n = dfs[(dfs['CreateDate'] >= start_date) & (dfs['CreateDate'] < end_date)].reset_index(drop=True)

#Filter all notes to include initial notes only. BIDMC will contain initial notes only. 
complete_df_initial = complete_df_n[complete_df_n['NoteTypeFull'].str.lower() == 'initial note'].reset_index(drop=True)
complete_df_initial.to_csv("complete_df_initial.csv", index=False)
complete_df_initial=pd.read_csv("complete_df_initial.csv")


In [4]:
#Read icdPlus data for BIDMC and convert AdmissionDate to datetime.
icdPlus = pd.read_csv('/home/gregory178/Desktop/NAX project/NAX_SDH/patientIDs_ICD_plus_SDH_BI.csv')
icdPlus['AdmissionDate'] = pd.to_datetime(icdPlus['AdmissionDate'], format='%Y-%m-%d')

#Ensure BDSPPatientID is converted to int64 if needed.
icdPlus['BDSPPatientID'] = icdPlus['BDSPPatientID'].astype('int64')

#Ensure CreateDate is in datetime.
complete_df_initial['CreateDate'] = pd.to_datetime(complete_df_initial['CreateDate'], errors='coerce')

#Check for any dates that were not parsed correctly.
invalid_dates = complete_df_initial[complete_df_initial['CreateDate'].isnull()]
if not invalid_dates.empty:
    print(f"Warning: Some dates in CreateDate could not be parsed:\n{invalid_dates}")

#Merge complete_df_initial with icdPlus for BIDMC on BDSPPatientID.
final_cpt_pos_notes = pd.merge(complete_df_initial, icdPlus, on='BDSPPatientID', how='inner')

#Calculate the absolute difference in days between CreateDate and AdmissionDate.
date_difference = abs((final_cpt_pos_notes['CreateDate'] - final_cpt_pos_notes['AdmissionDate']).dt.days)

#Filter the DataFrame to include only rows where the difference is less than 30 days to ensure ICD code corresponds with CreateDate.
ICDPositiveDS = final_cpt_pos_notes[date_difference < 30].reset_index(drop=True)

#Save the filtered DataFrame to CSV.
ICDPositiveDS.to_csv("BI_ICD_Positive_DS_before_500.csv", index=False)
ICDPositiveDS = pd.read_csv('/home/gregory178/Desktop/NAX project/NAX_SDH/BI_ICD_Positive_DS_before_500.csv')

In [5]:
#The following code connects BDSPPatientID to their note text and filters only to include examples that are longer than 500 words. 
def select_random_note_zip(df, patient_id, base_zip_path):
    patient_notes = df[df['BDSPPatientID'] == patient_id]
    valid_notes = []
    zip_file = None

    for _, row in patient_notes.iterrows():
        year = row['DeidentifiedName'].split('_')[-1][:4]
        file_path = f'{year}/' + row['DeidentifiedName']
        zip_path = f'{base_zip_path}bidmc_notes_{year}.zip'
        try:
            if zip_file is None or zip_file.filename != zip_path:
                if zip_file is not None:
                    zip_file.close()
                zip_file = zipfile.ZipFile(zip_path, 'r')
            
            with zip_file.open(file_path) as file_obj:
                try:
                    contents = file_obj.read().decode('utf-8')
                except UnicodeDecodeError:
                    try:
                        contents = file_obj.read().decode('latin1')
                    except UnicodeDecodeError:
                        contents = file_obj.read().decode('iso-8859-1')

                #Here we sort through based on word count. 
                word_count = len(contents.split())
                print(f"Length of contents for {file_path}: {word_count}")
                if word_count >= 500:
                    valid_notes.append((row['BDSPPatientID'], contents, row['CreateDate'], word_count, row['DeidentifiedName'], 'BIDMC'))
        except (KeyError, FileNotFoundError, zipfile.BadZipFile) as e:
            print(f"Error processing file {file_path}: {str(e)}")
            continue
    if zip_file is not None:
        zip_file.close()
    if valid_notes:
        #Randomly select one note from notes longer than 500 words.
        selected_note = random.choice(valid_notes) 
        return selected_note
    else:
        return None

base_zip_path= '/media/gregory178/Elements/Zip_Files_BI/'
ICDPL_df=pd.read_csv('/home/gregory178/Desktop/NAX project/NAX_SDH/BI_ICD_Positive_DS_before_500.csv')
FULLICDPL_list = ICDPL_df['BDSPPatientID'].tolist() 
columns = ['BDSPPatientID', 'note_text', 'CreateDate', 'length', 'deidentifiedname', 'hospital']

#Making ICD + and - groups for MGB
ICDPL_BI = pd.DataFrame()
ICDMI_BI = pd.DataFrame()



In [6]:
#Now we are making the ICD+ csv for initial notes.
counterPL = 0
added_patient_idsPL = []
print(len(added_patient_idsPL))
print(added_patient_idsPL)

#Use a random seed to ensure reproducibility
random.seed(2024)

while len(ICDPL_BI) < 750:
    randpatientID = random.choice(FULLICDPL_list) 
    UPDATED_LEN_GROUP=len(FULLICDPL_list)
    print(f"Updated length list {UPDATED_LEN_GROUP}.")
    if randpatientID not in added_patient_idsPL:
        rnote2 = select_random_note_zip(complete_df_initial, randpatientID, base_zip_path)
        if rnote2 is not None:
            ICDPL_BI = pd.concat([ICDPL_BI, pd.DataFrame([rnote2])], ignore_index=True) 
            added_patient_idsPL.append(randpatientID)
            FULLICDPL_list.remove(randpatientID)
            counterPL += 1
            print(f"Added note for patient {randpatientID}. Total notes added: {counterPL}")
        else:
            print(f"No valid notes for patient {randpatientID}")
            FULLICDPL_list.remove(randpatientID)
    else:
        print(f"Patient {randpatientID} already has a note added")
        FULLICDPL_list.remove(randpatientID)
    if not FULLICDPL_list:
        print("No more patient IDs left to process")

ICDPL_BI.columns = ['BDSPPatientID', 'NoteContent', 'ContactDate', 'WordCount', 'NoteFileName', 'Site']
ICDPL_BI.to_csv('BI_sampling_cohort_ICD+_initial_notes.csv', index=False)

0
[]
Updated length list 52168.
Length of contents for 2019/Notes_1131154161_471146795_20190924.txt: 929
Length of contents for 2019/Notes_1131154161_3349810315_20190928.txt: 155
Length of contents for 2019/Notes_1131154161_3349810317_20190928.txt: 1384
Length of contents for 2019/Notes_1131154161_471146802_20190926.txt: 286
Added note for patient 151295293. Total notes added: 1
Updated length list 52167.
Length of contents for 2013/Notes_1129913022_530864983_20130726.txt: 1240
Length of contents for 2013/Notes_1129913022_530864986_20130816.txt: 114
Length of contents for 2013/Notes_1129913022_530864993_20131005.txt: 1166
Length of contents for 2018/Notes_1129913022_530865034_20180623.txt: 1408
Length of contents for 2019/Notes_1129913022_530865048_20191203.txt: 1995
Length of contents for 2019/Notes_1129913022_530865051_20191208.txt: 1303
Length of contents for 2019/Notes_1129913022_530865055_20191214.txt: 747
Length of contents for 2019/Notes_1129913022_530865056_20191214.txt: 374
Le

In [7]:
#Here we are creating the ICD minus sampling cohort.
icd_minus_df = pd.read_csv('patientIDs_ICD_minus2_SDH_BI.csv')
full_patient_list = sorted( set(complete_df_initial.BDSPPatientID) & set(icd_minus_df.BDSPPatientID) )
FULLICDMI_list = sorted( set(full_patient_list) - set(FULLICDPL_list) )   

In [8]:
#Here we are making the ICD minus csv with initial notes.
counterMI = 0
added_patient_idsMI = []
print(len(added_patient_idsMI))
print(added_patient_idsMI)
random.seed(13)

while len(ICDMI_BI) < 750:
    randpatientID = random.choice(FULLICDMI_list) 
    UPDATED_LEN_GROUP=len(FULLICDMI_list)
    print(f"Updated length list {UPDATED_LEN_GROUP}.")
    if randpatientID not in added_patient_idsMI:
        rnote2 = select_random_note_zip(complete_df_initial, randpatientID, base_zip_path)
        if rnote2 is not None:
            ICDMI_BI = pd.concat([ICDMI_BI, pd.DataFrame([rnote2])], ignore_index=True) 
            added_patient_idsMI.append(randpatientID)
            FULLICDMI_list.remove(randpatientID)
            counterMI += 1
            print(f"Added note for patient {randpatientID}. Total notes added: {counterMI}")
        else:
            print(f"No valid notes for patient {randpatientID}")
            FULLICDMI_list.remove(randpatientID)
    else:
        print(f"Patient {randpatientID} already has a note added")
        FULLICDMI_list.remove(randpatientID)
    if not FULLICDMI_list:
        print("No more patient IDs left to process")

ICDMI_BI.columns = ['BDSPPatientID', 'NoteContent', 'ContactDate', 'WordCount', 'NoteFileName', 'Site']
ICDMI_BI.to_csv('BI_sampling_cohort_ICD_minus_initial_notes.csv', index=False)

0
[]
Updated length list 305520.
Length of contents for 2019/Notes_1130706045_338115816_20191127.txt: 918
Added note for patient 150847210. Total notes added: 1
Updated length list 305519.
Length of contents for 2017/Notes_1130495152_1142370651_20170406.txt: 601
Length of contents for 2017/Notes_1130495152_249810042_20170105.txt: 466
Length of contents for 2020/Notes_1130495152_1142370661_20200727.txt: 597
Length of contents for 2020/Notes_1130495152_1142370663_20201012.txt: 1172
Length of contents for 2020/Notes_1130495152_1142370664_20201012.txt: 364
Length of contents for 2020/Notes_1130495152_1142370669_20201015.txt: 1827
Length of contents for 2020/Notes_1130495152_1142370682_20201017.txt: 791
Error processing file 2022/Notes_1130495152_1142370693_20221213.txt: "There is no item named '2022/Notes_1130495152_1142370693_20221213.txt' in the archive"
Added note for patient 150636640. Total notes added: 2
Updated length list 305518.
Length of contents for 2010/Notes_1130314567_1932662